# Define Audit Process

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [1]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [9]:
select top 6 * from adlste_coa.coa_audit_log order by 1 desc

Run_ID,ParentRun_ID,Site_ID,TableName,StartDate,EndDate,CallingSP,SQLVersion,Run_Type,Run_Status,Run_Message,Run_User,Run_TS
97,96,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,PK151002,2020-06-11 17:34:16
96,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,PK151002,2020-06-11 17:34:13
95,94,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,PK151002,2020-06-11 15:31:07
94,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,PK151002,2020-06-11 15:31:07
93,92,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,PK151002,2020-06-11 15:05:14
92,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,PK151002,2020-06-11 15:05:13


In [ ]:
select top 10 * from adlste_coa.coa_audit_summary order by 1 desc

In [3]:
call adlste_coa.sp_meta_objects_refresh('none');

Success: 0 rows affected

In [4]:
Select * from adlste_coa.coa_meta_objects
where Tablename  like '%mat%'
order by ObjType, ProcessName,  ObjType desc;

DatabaseName,TableName,TableKind,ObjType,ObjFunction,ProcessName
ADLSTE_COA,coa_mat_csi_cod,V,coa,mat,csi_cod
ADLSTE_COA,coa_mat_csi_node_day,V,coa,mat,csi_node_day
ADLSTE_COA,coa_mat_csi_siteid_day,V,coa,mat,csi_siteid_day
ADLSTE_COA,coa_mat_csi_siteid_node_day,V,coa,mat,csi_siteid_node_day
ADLSTE_COA,coa_mat_gtm_customer,V,coa,mat,gtm_customer
ADLSTE_COA,coa_mat_oae_array_day,V,coa,mat,oae_array_day
ADLSTE_COA,coa_mat_sales_contact,V,coa,mat,sales_contact
ADLSTE_COA,coa_mat_sales_role_qlid,V,coa,mat,sales_role_qlid
ADLSTE_COA,coa_mat_shc_node_count3,V,coa,mat,shc_node_count3
ADLSTE_COA,coa_mat_shc_siteid_day_Coverage,V,coa,mat,shc_siteid_day_Coverage


<br/><br/><br/>
## Audit_Run (sub-stored_procedure)
### for tracking audit columns per row while minimizing storage
- database:    adlste_coa
- table:       coat_audit_run
- view:        coa_audit_run
- stored proc: sp_audit_run

In [9]:
drop table adlste_coa.coat_audit_runlog

Success: 40 rows affected

In [10]:
Create multiset table adlste_coa.coat_audit_runlog
(
 Run_ID       INTEGER 
,ParentRun_ID INTEGER
,Site_ID      VARCHAR(128)
,TableName    VARCHAR(128)
,StartDate    DATE
,EndDate      DATE
,CallingSP    VARCHAR(128)
,SQLVersion   VARCHAR(128)
,Run_Type     VARCHAR(128)
,Run_Status   VARCHAR(128)
,Run_Message  VARCHAR(512)
,Run_User     VARCHAR(128) Default USER
,Run_TS       Timestamp(0) Default Current_Timestamp(0)
) 
primary index (Site_ID, TableName);

Success: 0 rows affected

In [11]:
replace view adlste_coa.coa_audit_runlog 
as locking row for access
Select * from adlste_coa.coat_audit_runlog

Success: 13 rows affected

In [12]:
replace PROCEDURE adlste_coa.sp_audit_log (inout runid integer
                                              ,in siteid varchar(128)
                                              ,in tblname varchar(128)
                                              ,in callingsp varchar(128)
                                              ,in sqlversion varchar(128)
                                              ,in startdate date
                                              ,in enddate date
                                              ,in runtype varchar(128)
                                              ,in runstatus varchar(128)
                                              ,in runmessage varchar(512)
                                              )
SQL SECURITY CREATOR
BEGIN
DECLARE runidout INTEGER;

    SELECT coalesce(Max(Run_ID)+1,1) as Run_ID   INTO runidout 
    FROM adlste_coa.coat_audit_runlog 
    ;
    
    Insert into adlste_coa.coat_audit_runlog 
    values
    ( :runidout
     ,:runid --parentrunid
     ,:siteid
     ,:tblname
     ,:startdate
     ,:enddate
     ,:callingsp
     ,:sqlversion
     ,:runtype
     ,:runstatus
     ,:runmessage
     ,USER 
     ,Current_Timestamp(0)
    );
    
    SET runid = runidout ;

END;

Success: 0 rows affected

In [ ]:
-- delete from adlste_coa.coat_audit_runlog -- don't do this

### Example of some calls - designed to be embedded in other SPs

In [ ]:
-- start SP()
CALL adlste_coa.sp_audit_log(0
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'START' );  -- Run Status

In [ ]:
-- do stuff, but need a checkpoint 
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'CHECKPOINT' );  -- Run Status

In [ ]:
-- end of SP()
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'END'); -- Run Status

In [3]:
replace view adlste_coa.coa_audit_log as
locking row for access
Select * from adlste_coa.coat_audit_runlog

Success: 13 rows affected

In [4]:
replace view adlste_coa.coa_audit_summary   as locking row for access
Select 
 case when ParentRun_ID=0 then Run_ID else ParentRun_ID end as Run_ID
,Site_ID
,min(cast(Run_TS as DATE)) as LogDate
,TableName
,CallingSP
,SQLVersion
,Run_Type
,count(*) as Steps
,sum(case when Run_Status='ERROR' then 1 else 0 end) as Errors
,sum(case when Run_Status='WARNING' then 1 else 0 end) as Warnings
,min(case when Run_Status='START' then Run_TS else NULL end) as Start_TS
,max(case when Run_Status='END' then Run_TS else NULL end) as End_TS
from adlste_coa.coa_audit_runlog
Group by 1,2,4,5,6,7

Success: 13 rows affected

In [5]:
replace view adlste_coa.coa_audit_log_summary as
select * from adlste_coa.coa_audit_summary 

Success: 13 rows affected

In [7]:
select * from adlste_coa.coa_audit_log order  by 1 desc

Run_ID,ParentRun_ID,Site_ID,TableName,StartDate,EndDate,CallingSP,SQLVersion,Run_Type,Run_Status,Run_Message,Run_User,Run_TS
97,96,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,PK151002,2020-06-11 17:34:16
96,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,PK151002,2020-06-11 17:34:13
95,94,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,PK151002,2020-06-11 15:31:07
94,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,PK151002,2020-06-11 15:31:07
93,92,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,PK151002,2020-06-11 15:05:14
92,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-11,2020-06-11,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,PK151002,2020-06-11 15:05:13
91,90,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-10,2020-06-10,adlste_coa.sp_dat_dbcinfo,1.15,normal run,END,,ND186026,2020-06-10 15:20:43
90,0,trancend_source_01,adlste_coa.coat_dat_dbcinfo,2020-06-10,2020-06-10,adlste_coa.sp_dat_dbcinfo,1.15,normal run,START,,ND186026,2020-06-10 15:20:43
89,84,tdprd-na186009,adlste_coa.coat_dat_DBQL_core,2020-06-05,2020-06-09,adlste_coa.sp_dat_DBQL_core,1,normal run,END,,NA186009,2020-06-10 11:29:34
88,84,tdprd-na186009,adlste_coa.coat_dat_DBQL_core,2020-06-05,2020-06-09,adlste_coa.sp_dat_DBQL_core,1,normal run,STEP,Inserted stg into coat_dat_CPU,NA186009,2020-06-10 11:29:34


In [8]:
select * from adlste_coa.coa_audit_summary order  by 1 desc

Run_ID,Site_ID,LogDate,TableName,CallingSP,SQLVersion,Run_Type,Steps,Errors,Warnings,Start_TS,End_TS
96,trancend_source_01,2020-06-11,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-06-11 17:34:13,2020-06-11 17:34:16
94,trancend_source_01,2020-06-11,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-06-11 15:31:07,2020-06-11 15:31:07
92,trancend_source_01,2020-06-11,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-06-11 15:05:13,2020-06-11 15:05:14
90,trancend_source_01,2020-06-10,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-06-10 15:20:43,2020-06-10 15:20:43
84,tdprd-na186009,2020-06-10,adlste_coa.coat_dat_DBQL_core,adlste_coa.sp_dat_DBQL_core,1,normal run,6,0,0,2020-06-10 11:29:32,2020-06-10 11:29:34
82,trancend_source_01,2020-06-08,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-06-08 19:30:31,2020-06-08 19:30:32
80,TDCLOUD14TD03,2020-06-08,adlste_coa.coat_dat_top_users,adlste_coa.sp_dat_top_users,v1,normal run,2,0,0,2020-06-08 18:48:50,2020-06-08 18:48:51
78,trancend_source_01,2020-06-08,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-06-08 18:41:31,2020-06-08 18:41:32
76,TDCLOUD14TD03,2020-06-08,adlste_coa.coat_dat_top_users,adlste_coa.sp_dat_top_users,v1,normal run,2,0,0,2020-06-08 13:07:06,2020-06-08 13:07:06
74,TDCLOUD14TD03,2020-06-08,adlste_coa.coat_dat_top_users,adlste_coa.sp_dat_top_users,v1,normal run,2,0,0,2020-06-08 12:50:19,2020-06-08 12:50:19
